In [ ]:
#####################################################################################################
## This script contains the code to add transmembrane info to master table                         ##
## Mensah & Niskanen et al.                                                                        ##
## Aberrant phase separation and nucleolar dysfunction can underlie rare genetic diseases 2023     ##
## Author: Alexandre P Magalhaes                                                                   ##
#####################################################################################################

In [2]:
from __future__ import print_function
import re, gzip, time, itertools, io
import sys
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import Bio
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas_bokeh
from bokeh.io import output_file, show
pandas_bokeh.output_notebook()
import localcider
from localcider.sequenceParameters import SequenceParameters
from pandarallel import pandarallel
import swifter

Loading BokehJS ...

In [3]:
df = pd.read_csv('MasterTable_wNMD_wVariantDisease_wRK_WHidrophobic_wNLS_wTransmembrane_220505.csv',low_memory=False)
df.head()

,Uploaded_variation,Location,Feature,PepID,Gene,gene_symbol,Sequence,TrueIDRstart,SeqType,seqTypeExp,...,ID,Helix_Topology,NLS_score,NLS_position,NLS_sequence,Hydrophobic_Match,Hydrophobic_position,RK_Match,RK_position,Variant_omega
0,692054,16:86511252-86511259,ENST00000262426,ENSP00000262426,ENSG00000103241,FOXF1,MSSAPEKQQPPHGGGGGGGGGGGAAMDPASSGPSKAKKTNAGIRRP...,207,Frameshift,Frameshift,...,ENSP00000262426_A/X|692054|FOXF1,NaN,[0.708],[(244-255)],['PAAAHRRRWGHG'],['YSYIALIVMAIQS'],"[(50, 63)]",NaN,NaN,0.799188
1,1013621,21:34792484-34792490,ENST00000399240,ENSP00000382184,ENSG00000159216,RUNX1,MASDSIFESFPSYPQCFMRECILGMNPSRDVHDASTSRRFTPPSTA...,360,Frameshift,Frameshift,...,ENSP00000382184_R/X|1013621|RUNX1,NaN,"[0.974, 0.984]","[(381-398), (481-534)]","['RPTPARRKRREARSKPAR', 'PGLARLGPAGRRLRLRARGPPV...","['FESFPSYPQCFMRECIL', 'MSEALPLGAPDAGAALA', 'LP...","[(6, 23), (51, 68), (111, 126), (139, 153)]",['RRKRREARSKPARPP'],"[(385, 400)]",0.668419
2,rs1057519478,6:1611038-1611056,ENST00000645831,ENSP00000493906,ENSG00000054598,FOXC1,MQARYSVSSPNSLGVVPYLGGEQSYYRAAAAAAGGGYTAMPAPMSV...,141,Frameshift,Frameshift,...,ENSP00000493906_Q/X|rs1057519478|FOXC1,NaN,"[0.993, 0.996]","[(169-239), (270-298)]",['RRRRRFKKKDAVKDKEEKDRLHLKEPPPPGRRSRPTATRPVRSR...,"['YSYIALITMAIQNAPD', 'ITLNGIYQFIMDRFPFYRD', 'A...","[(80, 96), (98, 117), (252, 271)]",NaN,NaN,0.701037
3,rs1179926739,11:17720334-17720335,ENST00000250003,ENSP00000250003,ENSG00000129152,MYOD1,MELLSPPLRDVDLTAPDGSLCSFATTDDFYDDPCFDSPDLRFFEDL...,262,Frameshift,Frameshift,...,ENSP00000250003_R/X|rs1179926739|MYOD1,NaN,"[0.823, 0.603, 0.657]","[(186-202), (209-209), (214-221)]","['PGPRRRALQRRLRRVQP', 'R', 'QRPPERRP']",NaN,NaN,NaN,NaN,0.437444
4,rs1603388837,18:59269381,ENST00000256852,ENSP00000256852,ENSG00000134438,RAX,MHLPGCAPAMADGSFSLAGHLLRSPGGSTSRLHSIEAILGFTKDDG...,187,Frameshift,Frameshift,...,ENSP00000256852_S/X|rs1603388837|RAX,NaN,"[0.616, 0.648, 0.882]","[(61-65), (133-137), (245-283)]","['KERDR', 'KKKHR', 'SGRRCRAGAPRRCRACRASGRRRRAC...","['AGKVNLPEVRVQVWFQ', 'ARAAVAGRLGARCRW']","[(169, 185), (226, 241)]",NaN,NaN,0.424719


In [6]:
pandarallel.initialize(progress_bar=True, nb_workers = 40)

AA=list('FYW')

def add_omega(x):
    seq = x['Sequence']
    occ = seq.count('F') + seq.count('Y') + seq.count('W')
    if len(seq) > 20:
        b = round(len(seq)/occ)
        bsize = [b-1,b]
        if occ > 3:
            SeqOb = SequenceParameters(seq,blobsize=bsize)
            omega = SeqOb.get_kappa_X(AA)
            return omega
        else:
            return ''
    else:
        return ''
    
def add_omega2(x):
    seq = x['IDRseq']
    occ = seq.count('F') + seq.count('Y') + seq.count('W')
    if len(seq) >= 20 and occ > 3:
        b = round(len(seq)/occ)
        bsize = [b-1,b]
        SeqOb = SequenceParameters(seq,blobsize=bsize)
        omega = SeqOb.get_kappa_X(AA)
        return omega
    else:
        return ''
    
 

start = time.time()
#df['Variant_omega'] = df.parallel_apply(add_omega, axis=1)
df['Var_IDR_omega'] = df.parallel_apply(add_omega2, axis=1)
end = time.time()
print(end - start)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


EOFError: Ran out of input

In [7]:
df.head()

,Uploaded_variation,Location,Feature,PepID,Gene,gene_symbol,Sequence,TrueIDRstart,SeqType,seqTypeExp,...,Helix_Topology,NLS_score,NLS_position,NLS_sequence,Hydrophobic_Match,Hydrophobic_position,RK_Match,RK_position,Variant_omega,Var_IDR_omega
0,692054,16:86511252-86511259,ENST00000262426,ENSP00000262426,ENSG00000103241,FOXF1,MSSAPEKQQPPHGGGGGGGGGGGAAMDPASSGPSKAKKTNAGIRRP...,207,Frameshift,Frameshift,...,NaN,[0.708],[(244-255)],['PAAAHRRRWGHG'],['YSYIALIVMAIQS'],"[(50, 63)]",NaN,NaN,0.799188,
1,1013621,21:34792484-34792490,ENST00000399240,ENSP00000382184,ENSG00000159216,RUNX1,MASDSIFESFPSYPQCFMRECILGMNPSRDVHDASTSRRFTPPSTA...,360,Frameshift,Frameshift,...,NaN,"[0.974, 0.984]","[(381-398), (481-534)]","['RPTPARRKRREARSKPAR', 'PGLARLGPAGRRLRLRARGPPV...","['FESFPSYPQCFMRECIL', 'MSEALPLGAPDAGAALA', 'LP...","[(6, 23), (51, 68), (111, 126), (139, 153)]",['RRKRREARSKPARPP'],"[(385, 400)]",0.668419,0.263826
2,rs1057519478,6:1611038-1611056,ENST00000645831,ENSP00000493906,ENSG00000054598,FOXC1,MQARYSVSSPNSLGVVPYLGGEQSYYRAAAAAAGGGYTAMPAPMSV...,141,Frameshift,Frameshift,...,NaN,"[0.993, 0.996]","[(169-239), (270-298)]",['RRRRRFKKKDAVKDKEEKDRLHLKEPPPPGRRSRPTATRPVRSR...,"['YSYIALITMAIQNAPD', 'ITLNGIYQFIMDRFPFYRD', 'A...","[(80, 96), (98, 117), (252, 271)]",NaN,NaN,0.701037,0.467048
3,rs1179926739,11:17720334-17720335,ENST00000250003,ENSP00000250003,ENSG00000129152,MYOD1,MELLSPPLRDVDLTAPDGSLCSFATTDDFYDDPCFDSPDLRFFEDL...,262,Frameshift,Frameshift,...,NaN,"[0.823, 0.603, 0.657]","[(186-202), (209-209), (214-221)]","['PGPRRRALQRRLRRVQP', 'R', 'QRPPERRP']",NaN,NaN,NaN,NaN,0.437444,
4,rs1603388837,18:59269381,ENST00000256852,ENSP00000256852,ENSG00000134438,RAX,MHLPGCAPAMADGSFSLAGHLLRSPGGSTSRLHSIEAILGFTKDDG...,187,Frameshift,Frameshift,...,NaN,"[0.616, 0.648, 0.882]","[(61-65), (133-137), (245-283)]","['KERDR', 'KKKHR', 'SGRRCRAGAPRRCRACRASGRRRRAC...","['AGKVNLPEVRVQVWFQ', 'ARAAVAGRLGARCRW']","[(169, 185), (226, 241)]",NaN,NaN,0.424719,


In [8]:
df.to_csv('MasterTable_wNMD_wVariantDisease_wRK_WHidrophobic_wNLS_wTransmembrane_220505.csv', index=False)